In [ ]:
import os
import getpass

import pandas as pd
import matplotlib.pyplot as plt

import asyncio

from typing import Sequence

import autogen
from autogen_core.models import UserMessage
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMessageTermination, TimeoutTermination, MaxMessageTermination, StopMessageTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

In [2]:
# Load your API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [3]:
# Create OpenAI model client for inference
openai_model_client = OpenAIChatCompletionClient(
    model = "gpt-4o-mini",
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [4]:
# Define all model tools
async def search_web_tool(query: str) -> str:
    """Searches web for relevant information."""
    # Call API!
    return "The start was 10 and the end was 15."

def percentage_change_tool(start: float, end: float) -> float:
    """Finds the percentage change between start and end."""
    return ((end - start) / start) * 100

def addition_tool(first: float, second: float) -> float:
    """Adds two numbers together."""
    return first + second

In [5]:
user_agent = UserProxyAgent(
    name = "UserProxyAgent",
    description = "A user-facing agent to receive input.",
    input_func = input,
)

web_search_agent = AssistantAgent(
    name = "WebSearchAgent",
    model_client = openai_model_client,
    description = "A web search agent for relevant results.",
    tools = [search_web_tool],
    system_message = """
        You are a web search agent.
        Your only tool is search_web_tool - use it to find information.
        You make only one search call at a time.
        Once you have the results, you never do calculations based on them.
    """
)

data_manipulator_agent = AssistantAgent(
    name = "DataManipulatorAgent",
    model_client = openai_model_client,
    description = "A data manipulator agent. Useful for performing calculations and ranking items according to relevance.",
    tools = [percentage_change_tool, addition_tool],
    system_message = """
        You are an expert data manipulator agent.
        Given the tasks you have been assigned, you should analyze the data and provide results.
    """
)

writer_agent = AssistantAgent(
    name = "WriterAgent",
    model_client = openai_model_client,
    description = "A writer agent. All writing should be in English, make sense, and be perfectly clear.",
    system_message = """
        You are an expert writer, on par with the best in the world.
        Given the data that you have received, you should write a full, cohesive, and ranked report about it that is directly in line with the user's request.
        You should emphasize and display adherence to all user filters that have been requested.
    """
)

In [6]:
# Set up the multi-agent team, rotation, and termination conditions

text_mention_termination = TextMessageTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages = 25)
timeout_termination = TimeoutTermination(timeout_seconds = 10)
stop_termination = StopMessageTermination()

tm_termination = text_mention_termination | max_messages_termination

team = RoundRobinGroupChat(
    participants = [user_agent, web_search_agent, data_manipulator_agent, writer_agent], 
    termination_condition = tm_termination
)

In [ ]:
# Run a task through the multi-agent system
await team.reset()

task = "Do whatever the UserProxyAgent desires."

await Console(team.run_stream(task = task))

---------- user ----------
Do whatever the UserProxyAgent desires.
---------- UserProxyAgent ----------
APPROVE
---------- WebSearchAgent ----------
[FunctionCall(id='call_8KRKrQcpAf5bc3BryyC5bbQk', arguments='{"query":"latest news on technology and innovation"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='The start was 10 and the end was 15.', name='search_web_tool', call_id='call_8KRKrQcpAf5bc3BryyC5bbQk', is_error=False)]
---------- WebSearchAgent ----------
The start was 10 and the end was 15.
---------- DataManipulatorAgent ----------
[FunctionCall(id='call_c3bDGmB0OVI4M37cl5HrPpvh', arguments='{"start":10,"end":15}', name='percentage_change_tool')]
---------- DataManipulatorAgent ----------
[FunctionExecutionResult(content='50.0', name='percentage_change_tool', call_id='call_c3bDGmB0OVI4M37cl5HrPpvh', is_error=False)]
---------- DataManipulatorAgent ----------
50.0
---------- WriterAgent ----------
### Report on User Input Func